<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Settings

<div class="alert alert-success" role="info">
    <p><b>Note:</b> this data product framework is developed by Naas open source community. You can <a href="https://github.com/sponsors/jupyter-naas" target="_blank">sponsor us</a> if you find it usefull.
</p>
</div>

This notebook allows you to configure and run your data product on your Naas server. *Create an account on [Naas.ai](https://www.naas.ai/free-forever) if you don't already have one.*

Please follow the step below to setup this data product.

## Input

### Install requirements
Install the packages required to run the product.

In [ ]:
!pip install --user --upgrade -r "requirements.txt"

### Import libraries

In [ ]:
import naas_data_product
import naas
from naas_drivers import gsheet
import glob
import os
from IPython.display import clear_output
from tqdm import tqdm

### Setup variables
**Pre-requisite**
- Duplicate our Google Sheets spreadsheet model
- Share your Google Sheets spreadsheet with our service account : 🔗 naas-share@naas-gsheets.iam.gserviceaccount.com

**Inputs**
- `abi_spreadsheet`: Google Sheets spreadsheet URL
- `sheet_entity`: Entity sheet name that stores all your personal data

**Outputs**
- `output_dir_plugin`: Cookie used to authenticate Members and API clients.

In [ ]:
# Inputs
abi_spreadsheet = naas.secret.get("ABI_SPREADSHEET")
if abi_spreadsheet is None:
    abi_spreadsheet = input("Enter your Google Sheets URL:")
    naas.secret.add("ABI_SPREADSHEET", abi_spreadsheet)
sheet_entity = "ENTITY"

# Outputs
output_dir = os.path.join("outputs", "entity")
output_dir_plugin = os.path.join("outputs", "plugins")

## Model

### Get data from Google Sheet spreadsheet

In [ ]:
df_gsheet = gsheet.connect(abi_spreadsheet).get(sheet_name=sheet_entity).fillna("NA")
print("Rows:", len(df_gsheet))
df_gsheet.head(1)

### Store LinkedIn URL
The LinkedIn URL will be used in content and growth engine.

In [ ]:
linkedin_url = df_gsheet.loc[0, "LINKEDIN_URL"]
print("LinkedIn URL:", linkedin_url)
pdump(output_dir, linkedin_url, "linkedin_url")
naas.dependency.add(os.path.join(output_dir, "linkedin_url.pickle"))

### Store Emails
The emails will be used for the notifications

In [ ]:
emails = df_gsheet.loc[0, "EMAILS"]
print("Emails:", emails)
pdump(output_dir, emails, "emails")
naas.dependency.add(os.path.join(output_dir, "emails.pickle"))

### Store Prompt ICP
The prompt will be used in the growth engine.

In [ ]:
prompt_icp = df_gsheet.loc[0, "PROMPT_ICP"]
print("Prompt ICP:", prompt_icp)
pdump(output_dir, prompt_icp, "prompt_icp")
naas.dependency.add(os.path.join(output_dir, "prompt_icp.pickle"))

### Store Sales Messagings
The prompt will be used in the sales engine to create sales messagins options on new sales qualified leads.

In [ ]:
prompt_sales_messagings = df_gsheet.loc[0, "PROMPT_SALES_MESSAGINGS"]
print("Prompt Sales Messagings:", prompt_sales_messagings)
pdump(output_dir, prompt_sales_messagings, "prompt_sales_messagings")
naas.dependency.add(os.path.join(output_dir, "prompt_sales_messagings.pickle"))

### Add secrets to Naas

In [ ]:
for x in ["LINKEDIN_LI_AT", "LINKEDIN_JSESSIONID", "OPENAI_API_KEY"]:
    value = df_gsheet.loc[0, x]
    if value != "NA":
        print(f"Secret '{x}' to be added:")
        naas.secret.add(x, value)

## Output

### Run plugins

In [ ]:
# List plugin files
files = glob.glob("*/**/__plugin__.ipynb", recursive=True)

# Execute notebook
for file in tqdm(files):
    print(f"File: {file}")
    print()
    %run $file
    clear_output()

### Get plugins URL

In [ ]:
data = []
plugins = glob.glob(f"{output_dir_plugin}/*.json")

for plugin in plugins:
    plugin_url = naas.asset.find(plugin)
    data.append(
        {
            "name": plugin.split("/")[-1].replace(".json", ""),
            "image": "",
            "url": plugin_url,
            "new_chat_url": f"https://naas.ai/chat/use?plugin_url={plugin_url}"
        }
    )
    
data